In [1]:
import warnings
import re
import math
import json
import glob
import random
from collections import Counter, defaultdict

import pathlib
import tqdm
from pathlib import Path
import numpy as np
from joblib import Parallel, delayed
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix, csr_array, lil_array, lil_matrix, save_npz, load_npz

pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

# Prepare data

In [2]:
games_path = "/home/artermiloff/Datasets/Steam/SteamGames/games_clean.csv"
games_reviews_path = "/home/artermiloff/Datasets/Steam/SteamReviewsCombined/*.csv"
steam_users_path = "/home/artermiloff/Datasets/Steam/steam_users.csv"

In [3]:
# games information
game_info_df = pd.read_csv(games_path)
game_info_df["TotalReviews"] = game_info_df["Positive"] + game_info_df["Negative"]
# remove trash games
game_info_df = game_info_df[~game_info_df["Estimated owners"].isin(['0 - 0', '0 - 20000'])]
game_info_df = game_info_df.query("TotalReviews >= 400")
popular_games = set(game_info_df['AppID'])
len(popular_games)

8144

In [4]:
similar_games_df = pd.read_csv("/home/artermiloff/Datasets/Steam/steam_game_similarities.csv")

# Create sparse interaction matrix

In [5]:
appids, steamids, votes = [], [], []
review_files = glob.glob(games_reviews_path)
for review_file in tqdm.tqdm(review_files):
    appid = int(Path(review_file).stem)
    if appid not in popular_games:
        continue
    reviews = pd.read_csv(review_file)
    reviews["AppID"] = appid
    appids += [appid] * reviews.shape[0]
    steamids += list(reviews["author_steamid"])
    votes += list(2 * reviews["voted_up"].astype(int) - 1) # convert vote up/down to 1 and -1 respectively

100%|████████████████████████████████████| 71148/71148 [04:04<00:00, 290.54it/s]


In [6]:
MIN_REVIEWS = 5
MAX_REVIEWS = 250

steamids_num_reviews = sorted(Counter(steamids).items(), key=lambda item: item[1], reverse=True)

valid_steamids_list = [steamid for steamid, num_reviews in steamids_num_reviews \
                      if MIN_REVIEWS <= num_reviews <= MAX_REVIEWS]
valid_steamids = set(valid_steamids_list)
len(valid_steamids)

4304315

In [7]:
# remove all users who have too little (< 5) or too many (> 250) reviews
appids_filtered, steamids_filtered, votes_filtered = [], [], []
for i in tqdm.tqdm(range(len(steamids))):
    if steamids[i] in valid_steamids:
        appids_filtered.append(appids[i])
        steamids_filtered.append(steamids[i])
        votes_filtered.append(votes[i])
        
appids, steamids, votes = appids_filtered, steamids_filtered, votes_filtered

100%|████████████████████████| 100854237/100854237 [00:24<00:00, 4078307.53it/s]


In [8]:
STEAMID_TO_INDEX = {steamid: i for i, steamid in enumerate(set(steamids))}
APPID_TO_INDEX = {appid: i for i, appid in enumerate(set(appids))}

INDEX_TO_STEAMID = {v: k for k, v in STEAMID_TO_INDEX.items()}
INDEX_TO_APPID = {v: k for k, v in APPID_TO_INDEX.items()}

remapped_steamids = [STEAMID_TO_INDEX[steamid] for steamid in steamids]
remapped_appids = [APPID_TO_INDEX[appid] for appid in appids]

sparse_interaction_matrix = csr_matrix((votes, (remapped_steamids, remapped_appids)),
                                      shape=(max(remapped_steamids) + 1, max(remapped_appids) + 1))
display(sparse_interaction_matrix, Counter(sparse_interaction_matrix.data))

<4304315x8139 sparse matrix of type '<class 'numpy.int64'>'
	with 46732680 stored elements in Compressed Sparse Row format>

Counter({1: 40174856, -1: 6557824})

In [9]:
# # Сохранение разреженной матрицы в файл
# save_npz('/home/artermiloff/Datasets/Steam/sparse_interaction_matrix.npz', sparse_interaction_matrix)

# Create train/test split

In [10]:
TEST_SIZE = 0.3

In [11]:
def flatten(xss):
    return [x for xs in xss for x in xs]


def train_test_split(interaction_matrix, test_size, index_to_appid, seed=42) -> (csr_matrix, dict):
    train_cols, train_rows, train_reviews = [], [], []
    test_interactions = dict()
    np.random.seed(seed)
    for i in tqdm.tqdm(range(interaction_matrix.shape[0])):
        game_idxs = interaction_matrix[i, :].indices
        reviews = interaction_matrix[i, :].data
        pos_reviews_idxs, neg_reviews_idxs = list(game_idxs[reviews == 1]), list(game_idxs[reviews == -1])

        user_test_size = int(test_size * len(reviews))
        if len(pos_reviews_idxs) < user_test_size:
            train_cols.append(game_idxs)
            train_rows.append([i] * len(game_idxs))
            train_reviews.append(reviews)
            continue
        
        np.random.shuffle(pos_reviews_idxs)

        test_interactions[i] = [index_to_appid[idx] for idx in pos_reviews_idxs[:user_test_size]]

        user_train_cols = pos_reviews_idxs[user_test_size:] + neg_reviews_idxs
        user_train_rows = [i] * len(user_train_cols)
        user_train_reviews = [1] * (len(pos_reviews_idxs) - user_test_size) + [-1] * len(neg_reviews_idxs)

        train_cols.append(user_train_cols)
        train_rows.append(user_train_rows)
        train_reviews.append(user_train_reviews)
        
    return csr_matrix((flatten(train_reviews), (flatten(train_rows), flatten(train_cols))),
                                  shape=interaction_matrix.shape), test_interactions

In [12]:
train_interaction_matrix, test_interactions = train_test_split(sparse_interaction_matrix, 
                                                                test_size=TEST_SIZE,
                                                               index_to_appid=INDEX_TO_APPID,
                                                               seed=42)

100%|██████████████████████████████| 4304315/4304315 [04:22<00:00, 16385.47it/s]


In [13]:
train_interaction_matrix

<4304315x8139 sparse matrix of type '<class 'numpy.int64'>'
	with 34846951 stored elements in Compressed Sparse Row format>

# Algorithms

In [14]:
def user_user_collaborative_filtering(interaction_matrix, user_index, k_neighbors=100, num_results=None):    
    # Calculate the user similarity matrix
    user_row = interaction_matrix[[user_index], :]
    user_similarity = cosine_similarity(user_row, interaction_matrix)
    user_similarity[0][user_index] = 0
    
    # Get the user's k nearest neighbors
    user_neighbors = np.argsort(user_similarity[0])[::-1][:k_neighbors]
    
    # Calculating like ratios and number of reviews for each game among neighbors
    user_ratings = []
    for i in range(interaction_matrix[user_neighbors].shape[1]):
        game_ratings = interaction_matrix[user_neighbors][:, i]
        
        # If at least one neighbor rated the game, calculate liked ratio
        # else, assign it 0 
        neighbor_average_rating = 0
        num_neighbors_reviewed = game_ratings.data.shape[0]
        if num_neighbors_reviewed != 0:
            c = Counter(game_ratings.data)
            pos = c[1] if 1 in c else 0
            neg = c[-1] if -1 in c else 0
            neighbor_average_rating = pos / (pos + neg)
        user_ratings.append((num_neighbors_reviewed, round(neighbor_average_rating, 2)))
    user_ratings = np.array(user_ratings) 
    
    selected_games = np.where(interaction_matrix[user_index].todense() == 0)[1]
    user_ratings = list(map(tuple, user_ratings[selected_games]))
    # Sort by most reviews, then by like ratio
    recommendations = sorted(zip(selected_games, user_ratings), key=lambda x: x[1], reverse=True)
    if num_results is not None:
        recommendations = recommendations[:num_results]
        
    recommendations = [i[0] for i in recommendations]
        
    return recommendations

In [15]:
# this fast implementation requires precomputing similar games (items) for each game (see TODO)
def item_item_collaborative_filtering(similar_games_df, user_played_appids, num_results):
    user_played_appids = set(user_played_appids)
    similar_games_for_user = similar_games_df[similar_games_df.appid.isin(user_played_appids)]
    similar_unplayed_games = similar_games_for_user[~similar_games_for_user.similar_appid.isin(
        user_played_appids)].sort_values("cos_distance", ascending=False).drop_duplicates(
        subset=["similar_appid"], keep="first")
    
    return similar_unplayed_games.similar_appid.head(num_results).values

In [16]:
def get_popular_recommendations(game_info_df, num_results):
    game_info_df = game_info_df.copy()
    game_info_df["Rating"] = (game_info_df["Positive"] / (
        game_info_df["Positive"] + game_info_df["Negative"])).round(2)
    top_games = game_info_df.sort_values(["Rating", "TotalReviews"], ascending=False).head(num_results)
    recommendations = list(top_games["AppID"])
    
    return recommendations

In [17]:
def get_new_recommendations(game_info_df, num_results):
    game_info_df = game_info_df.copy()
    game_info_df["Rating"] = (game_info_df["Positive"] / (
        game_info_df["Positive"] + game_info_df["Negative"])).round(2)
    game_info_df["Year"] = game_info_df['Release date'].astype(str).str[-4:]
    top_games = game_info_df.sort_values(["Year", "Rating", "TotalReviews"], ascending=False).head(num_results)
    recommendation = list(top_games["AppID"])
    
    return recommendation

# Metrics

In [18]:
def precision_at_k(actual, predicted, k):
    actual_set = set(actual)
    num_actual = len(actual_set)
    if num_actual < k:
        k = num_actual
        
    num_found = len(actual_set.intersection(set(predicted[:k])))
    precision = num_found / k
    
    return precision


def average_precision_at_k(actual, predicted, k):
    actual_set = set(actual)
    num_actual = len(actual_set)
    if num_actual < k:
        k = num_actual
        
    cum_sum = ap = 0
    for i in range(k):
        if predicted[i] in actual_set:
            cum_sum += 1
            ap += cum_sum / (i + 1)
    
    return ap / k


def mean_average_precision_at_k(actuals, predicteds, k):
    return np.mean([average_precision_at_k(a, p, k) for a, p in zip(actuals, predicteds)])


def recall_at_k(actual, predicted, k):
    actual_set = set(actual)
    num_actual = len(actual_set)
    if num_actual < k:
        k = num_actual
        
    num_found = len(actual_set.intersection(set(predicted[:k])))
    recall = num_found / num_actual
    
    return recall


def ndcg_at_k(actual, predicted, k):
    actual_set = set(actual)
    k = min(k, len(actual))
    # Calculating IDCG (Ideal Discounted Cumulative Gain)
    idcg = (1 / (np.log2(np.arange(2, k + 2)))).sum()

    # Calculating DCG (Discounted Cumulative Gain)
    if len(predicted) < k:
        predicted = predicted + [-1] * (k - len(predicted))
    dcg = np.sum([1 / np.log2(i + 2) for i in range(k) if predicted[i] in actual_set])

    # Calculating nDCG
    return dcg / idcg if idcg != 0 else 0


def mean_ndcg_at_k(actuals, predicteds, k):
    return np.mean([ndcg_at_k(a, p, k) for a, p in zip(actuals, predicteds)])

# Algorithm Evaluation

In [19]:
def get_games_played(interaction_matrix, df, user_index, app_index_dict, verbose=False):
    rated_games = np.where(interaction_matrix[user_index].todense() != 0)[1]
    
    games_played = list()
    # see what games are recommended for the user
    for game_idx in rated_games:
        game_id = get_key(app_index_dict, game_idx)
        games_played.append(game_id)
        
        if verbose:
            print(df[df['AppID'] == game_id]['Name'].item())
        
    return games_played

In [20]:
K_VALUES = [1, 5, 10, 25, 50, 100]

In [21]:
random.seed(42)
NUM_SUBSAMPLE = 43_000
metrics_steamids_sample = [it for it in random.sample(valid_steamids_list, NUM_SUBSAMPLE
                                              ) if STEAMID_TO_INDEX[it] in test_interactions]
metrics_steamids_full = [it for it in valid_steamids_list if STEAMID_TO_INDEX[it] in test_interactions]

In [22]:
recommendations_user_user_cf = Parallel(28, verbose=1)(delayed(user_user_collaborative_filtering
                                                  )(train_interaction_matrix, STEAMID_TO_INDEX[steamid],
                                                           k_neighbors=100
                                              ) for steamid in metrics_steamids_sample)
recommendations_user_user_cf = [
    [INDEX_TO_APPID[ind] for ind in user_rec] for user_rec in recommendations_user_user_cf
]

[Parallel(n_jobs=28)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=28)]: Done 144 tasks      | elapsed:   19.6s
[Parallel(n_jobs=28)]: Done 394 tasks      | elapsed:   44.2s
[Parallel(n_jobs=28)]: Done 744 tasks      | elapsed:  1.3min
[Parallel(n_jobs=28)]: Done 1194 tasks      | elapsed:  2.0min
[Parallel(n_jobs=28)]: Done 1744 tasks      | elapsed:  2.9min
[Parallel(n_jobs=28)]: Done 2394 tasks      | elapsed:  4.0min
[Parallel(n_jobs=28)]: Done 3144 tasks      | elapsed:  5.2min
[Parallel(n_jobs=28)]: Done 3994 tasks      | elapsed:  6.6min
[Parallel(n_jobs=28)]: Done 4944 tasks      | elapsed:  8.2min
[Parallel(n_jobs=28)]: Done 5994 tasks      | elapsed:  9.9min
[Parallel(n_jobs=28)]: Done 7144 tasks      | elapsed: 11.7min
[Parallel(n_jobs=28)]: Done 8394 tasks      | elapsed: 13.8min
[Parallel(n_jobs=28)]: Done 9744 tasks      | elapsed: 16.0min
[Parallel(n_jobs=28)]: Done 11194 tasks      | elapsed: 18.4min
[Parallel(n_jobs=28)]: Done 12744 tasks    

In [23]:
recommendations_item_item_cf = Parallel(28, verbose=1)(delayed(item_item_collaborative_filtering
                                                  )(similar_games_df, 
    [INDEX_TO_APPID[i] for i in train_interaction_matrix[STEAMID_TO_INDEX[steamid], :].indices], 200
                                              ) for steamid in metrics_steamids_full)

[Parallel(n_jobs=28)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=28)]: Done 176 tasks      | elapsed:    0.7s
[Parallel(n_jobs=28)]: Done 3648 tasks      | elapsed:    2.2s
[Parallel(n_jobs=28)]: Done 14848 tasks      | elapsed:    6.2s
[Parallel(n_jobs=28)]: Done 29248 tasks      | elapsed:   10.7s
[Parallel(n_jobs=28)]: Done 46848 tasks      | elapsed:   16.9s
[Parallel(n_jobs=28)]: Done 67648 tasks      | elapsed:   23.5s
[Parallel(n_jobs=28)]: Done 91648 tasks      | elapsed:   30.2s
[Parallel(n_jobs=28)]: Done 118848 tasks      | elapsed:   39.3s
[Parallel(n_jobs=28)]: Done 126264 tasks      | elapsed:   46.4s
[Parallel(n_jobs=28)]: Done 132564 tasks      | elapsed:   52.8s
[Parallel(n_jobs=28)]: Done 139464 tasks      | elapsed:   59.7s
[Parallel(n_jobs=28)]: Done 146964 tasks      | elapsed:  1.1min
[Parallel(n_jobs=28)]: Done 155064 tasks      | elapsed:  1.3min
[Parallel(n_jobs=28)]: Done 163764 tasks      | elapsed:  1.4min
[Parallel(n_jobs=28)]: 

In [24]:
recommendations_popular = [get_popular_recommendations(game_info_df, num_results=200)
                          ] * len(metrics_steamids_full)

In [25]:
recommendations_new = [get_new_recommendations(game_info_df, num_results=200)] * len(metrics_steamids_full)

In [26]:
for alg_name, recommendations_list, steamids_list in [
    ("User-based Collaborative Filtering", recommendations_user_user_cf, metrics_steamids_sample),
    ("Item-based Collaborative Filtering", recommendations_item_item_cf, metrics_steamids_full),
    ("Popular Non-Personalized", recommendations_popular, metrics_steamids_full),
    ("New Non-Personalized", recommendations_new, metrics_steamids_full),
]:
    print(alg_name)
    ap_dict, ndcg_dict, re_dict, pr_dict = defaultdict(list), defaultdict(list), defaultdict(list), defaultdict(list)
    for steamid, predicted in tqdm.tqdm(zip(steamids_list, recommendations_list)):
        actual = test_interactions[STEAMID_TO_INDEX[steamid]]

        for k in K_VALUES:
            re_dict[k].append(recall_at_k(actual, predicted, k))
            pr_dict[k].append(precision_at_k(actual, predicted, k))
            ap_dict[k].append(average_precision_at_k(actual, predicted, k))
            ndcg_dict[k].append(ndcg_at_k(actual, predicted, k))
            
    for k in K_VALUES:
        print("K = ", k)
        print("mP@K\t", round(np.mean(ap_dict[k]), 3))
        print("mR@K\t", round(np.mean(ndcg_dict[k]), 3))
        print("mAP@K\t", round(np.mean(ap_dict[k]), 3))
        print("nDCG@K\t", round(np.mean(ndcg_dict[k]), 3))
    print()

User-based Collaborative Filtering


42731it [00:01, 26809.66it/s]


K =  1
mP@K	 0.087
mR@K	 0.087
mAP@K	 0.087
nDCG@K	 0.087
K =  5
mP@K	 0.056
mR@K	 0.072
mAP@K	 0.056
nDCG@K	 0.072
K =  10
mP@K	 0.053
mR@K	 0.07
mAP@K	 0.053
nDCG@K	 0.07
K =  25
mP@K	 0.053
mR@K	 0.07
mAP@K	 0.053
nDCG@K	 0.07
K =  50
mP@K	 0.053
mR@K	 0.07
mAP@K	 0.053
nDCG@K	 0.07
K =  100
mP@K	 0.053
mR@K	 0.07
mAP@K	 0.053
nDCG@K	 0.07

Item-based Collaborative Filtering


4279113it [03:10, 22511.56it/s]


K =  1
mP@K	 0.072
mR@K	 0.072
mAP@K	 0.072
nDCG@K	 0.072
K =  5
mP@K	 0.046
mR@K	 0.06
mAP@K	 0.046
nDCG@K	 0.06
K =  10
mP@K	 0.044
mR@K	 0.059
mAP@K	 0.044
nDCG@K	 0.059
K =  25
mP@K	 0.043
mR@K	 0.058
mAP@K	 0.043
nDCG@K	 0.058
K =  50
mP@K	 0.043
mR@K	 0.058
mAP@K	 0.043
nDCG@K	 0.058
K =  100
mP@K	 0.043
mR@K	 0.058
mAP@K	 0.043
nDCG@K	 0.058

Popular Non-Personalized


4279113it [02:29, 28644.72it/s]


K =  1
mP@K	 0.015
mR@K	 0.015
mAP@K	 0.015
nDCG@K	 0.015
K =  5
mP@K	 0.007
mR@K	 0.009
mAP@K	 0.007
nDCG@K	 0.009
K =  10
mP@K	 0.006
mR@K	 0.009
mAP@K	 0.006
nDCG@K	 0.009
K =  25
mP@K	 0.006
mR@K	 0.008
mAP@K	 0.006
nDCG@K	 0.008
K =  50
mP@K	 0.006
mR@K	 0.008
mAP@K	 0.006
nDCG@K	 0.008
K =  100
mP@K	 0.006
mR@K	 0.008
mAP@K	 0.006
nDCG@K	 0.008

New Non-Personalized


4279113it [02:29, 28633.57it/s]


K =  1
mP@K	 0.001
mR@K	 0.001
mAP@K	 0.001
nDCG@K	 0.001
K =  5
mP@K	 0.0
mR@K	 0.001
mAP@K	 0.0
nDCG@K	 0.001
K =  10
mP@K	 0.0
mR@K	 0.001
mAP@K	 0.0
nDCG@K	 0.001
K =  25
mP@K	 0.0
mR@K	 0.001
mAP@K	 0.0
nDCG@K	 0.001
K =  50
mP@K	 0.0
mR@K	 0.001
mAP@K	 0.0
nDCG@K	 0.001
K =  100
mP@K	 0.0
mR@K	 0.001
mAP@K	 0.0
nDCG@K	 0.001

